In [19]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
from scipy import ndimage
import random
from scipy import ndimage
from scipy.optimize import least_squares
from func import *
from scipy import stats
from matplotlib.colors import LogNorm
from multiprocessing import Pool,Process, cpu_count

In [2]:
def nan_free_data(filename):
    hdu = fits.open(filename)
    qso_data = hdu[0].data
    qso_error = hdu[1].data
    qso_header = hdu[0].header
    wavestart = qso_header['CRVAL3']
    wavint = qso_header['CD3_3']
    wave = wavestart+np.arange(qso_data.shape[0])*wavint#. This is the same as the one below.
    qso_data[np.isnan(qso_data)] = 0.0000001
    qso_error[np.isnan(qso_error)] = 0.000001
    return qso_data,qso_error,wave,qso_header

In [3]:
def BIC(data,model,n_free_par):
    N = len(data)
    BIC=stats.chisquare(data,model)[0] +n_free_par*(np.log(N) - np.log(2*np.pi))
    return BIC

In [4]:
def FOV_modeling(obj,z,wave,mini_data,mini_error,BIC_cutoff,MC_loops):
    k = 1+z
    c = 300000
    central_vel = c*z
    
    lower_bounds = [0,0,central_vel - 1000,0,0,0,central_vel - 1000,0,-np.inf,-np.inf]
    upper_bounds = [np.inf,np.inf,central_vel + 1000,600,np.inf,np.inf,central_vel + 1000,600,np.inf,np.inf]
    bounds_p_init = (lower_bounds,upper_bounds)
    
    par = np.zeros((10,mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
    par_err = np.zeros((10,mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
    fitted = np.zeros((np.shape(wave)[0],mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
    residuals = np.zeros((np.shape(wave)[0],mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
 
    select_OIII = (wave > 4997*k) & (wave <5017*k)
    select_Hb = (wave > 4851*k) & (wave <4871*k)

    for i in range(mini_data.shape[1]):
        for j in range(mini_data.shape[2]):
            x = wave  
            y = mini_data[:,i,j]
            y_err = mini_error[:,i,j]
        
            amp_Hb = np.max(y[select_Hb])
            amp_OIII = np.max(y[select_OIII])
 
            try:       #(popt1,pcov1) = leastsq(full_gauss2_eline,x0=[0.1,0.2,central_vel,100,0.01,0.03,central_vel - 100,100,-0.001,0.002],args = (x,y,y_err),maxfev=10000000)
                result1 = least_squares(full_gauss_eline,bounds=bounds_p_init,x0=[amp_Hb,amp_OIII,central_vel,100,0.0,0.0,central_vel,0,0.0001,0.002],args = (x,y,y_err),max_nfev=10000000)
                popt1 = result1['x']
                model1 = full_gauss_eline(popt1,x,y,y_err)*(y_err)+y
                BIC1 = BIC(y,model1,6)
                
                result2 = least_squares(full_gauss2_eline,bounds=(lower_bounds,upper_bounds),x0=[0.7*amp_Hb,0.7*amp_OIII,central_vel,100,0.3*amp_Hb,0.3*amp_OIII,central_vel - 100,100,0.0001,0.002],args = (x,y,y_err),max_nfev=10000000)
                popt2 = result2['x']
                model2 = full_gauss2_eline(popt2,x,y,y_err)*(y_err)+y
                BIC2 = BIC(y,model2,10)
            
                if BIC2 - BIC1 > BIC_cutoff and np.abs(popt2[7]) > np.abs(popt2[3]):
                    popt = popt2
                elif BIC2 - BIC1 > BIC_cutoff and np.abs(popt2[7]) < np.abs(popt2[3]):
                    popt = [popt2[4],popt2[5],popt2[6],popt2[7],popt2[0],popt2[1],popt2[2],popt2[3],popt2[8],popt2[9]]
                else:
                    popt = popt1
            except ValueError:
                popt = [1e-15,1e-15,central_vel,0,1e-15,1e-15,central_vel,0,0.00001,0.0002]
       
        par[:,i,j] = popt
        model = full_gauss2_eline(popt,x,y,y_err)*(y_err)+y
        fitted[:,i,j] = model      
        residuals[:,i,j] = mini_data[:,i,j] - fitted[:,i,j]
                
        parameters_MC = np.zeros((len(popt),MC_loops))
    
        for l in range(MC_loops):
            iteration_data = np.random.normal(y,y_err)
        
            amp_Hb_MC = np.max(iteration_data[select_Hb])
            amp_OIII_MC = np.max(iteration_data[select_OIII])
      
            try:       #(popt1,pcov1) = leastsq(full_gauss2_eline,x0=[0.1,0.2,central_vel,100,0.01,0.03,central_vel - 100,100,-0.001,0.002],args = (x,y,y_err),maxfev=10000000)
                result1_MC = least_squares(full_gauss_eline,bounds=bounds_p_init,x0=[amp_Hb_MC,amp_OIII_MC,central_vel,100,0.0,0.0,central_vel,0,0.0001,0.002],args = (x,iteration_data,y_err),max_nfev=10000000)
                popt1_MC = result1_MC['x']
                model1_MC = full_gauss_eline(popt1,x,iteration_data,y_err)*(y_err)+iteration_data
                BIC1_MC = BIC(iteration_data,model1,6)
            
                result2_MC = least_squares(full_gauss2_eline,bounds=(lower_bounds,upper_bounds),x0=[0.7*amp_Hb_MC,0.7*amp_OIII_MC,central_vel,100,0.3*amp_Hb_MC,0.3*amp_OIII_MC,central_vel - 100,100,0.0001,0.002],args = (x,y,y_err),max_nfev=10000000)
                popt2_MC = result2_MC['x']
                model2_MC = full_gauss2_eline(popt2,x,iteration_data,y_err)*(y_err)+iteration_data
                BIC2_MC = BIC(iteration_data,model2,10)
                
                if BIC2_MC - BIC1_MC > BIC_cutoff and np.abs(popt2_MC[7]) > np.abs(popt2_MC[3]):
                    popt_MC = popt2_MC
                elif BIC2_MC - BIC1_MC > BIC_cutoff and np.abs(popt2_MC[7]) < np.abs(popt2_MC[3]):
                    popt_MC = [popt2_MC[4],popt2_MC[5],popt2_MC[6],popt2_MC[7],popt2_MC[0],popt2_MC[1],popt2_MC[2],popt2_MC[3],popt2_MC[8],popt2_MC[9]]
                else:
                    popt_MC = popt1_MC
            except ValueError:
                popt_MC = [1e-15,1e-15,central_vel,0,1e-15,1e-15,central_vel,0,0.00001,0.0002]  
                
            parameters_MC[:,l]=popt_MC
        parameters_err = np.std(parameters_MC,1)
        par_err[:,i,j] = parameters_err
        print (i,j)
    return par,par_err,fitted,residuals 

In [5]:
def maps(obj,par,destination_path_cube='/Volumes/Seagate'):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(par[0,:,:],name='amp_Hb'))
    hdus.append(fits.ImageHDU(par[1,:,:],name='amp_OIII'))
    hdus.append(fits.ImageHDU(par[2,:,:],name='vel_OIII'))
    hdus.append(fits.ImageHDU(par[3,:,:],name='vel_sigma_OIII'))
    hdus.append(fits.ImageHDU(par[4,:,:],name='amp_Hb_br'))
    hdus.append(fits.ImageHDU(par[5,:,:],name='amp_OIII_br'))
    hdus.append(fits.ImageHDU(par[6,:,:],name='vel_OIII_br'))
    hdus.append(fits.ImageHDU(par[7,:,:],name='vel_sigma_OIII_br'))
    hdus.append(fits.ImageHDU(par[8,:,:],name='m'))
    hdus.append(fits.ImageHDU(par[9,:,:],name='c'))
    hdu = fits.HDUList(hdus)

    hdu.writeto('%s/%s Extended/subcube_par_%s_extended_double_gauss.fits'%(destination_path_cube,obj,obj),overwrite=True)

In [6]:
def err_maps(obj,par_err,destination_path_cube='/Volumes/Seagate'):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(par_err[0,:,:],name='amp_Hb'))
    hdus.append(fits.ImageHDU(par_err[1,:,:],name='amp_OIII'))
    hdus.append(fits.ImageHDU(par_err[2,:,:],name='vel_OIII'))
    hdus.append(fits.ImageHDU(par_err[3,:,:],name='vel_sigma_OIII'))
    hdus.append(fits.ImageHDU(par_err[4,:,:],name='amp_Hb_br'))
    hdus.append(fits.ImageHDU(par_err[5,:,:],name='amp_OIII_br'))
    hdus.append(fits.ImageHDU(par_err[6,:,:],name='vel_OIII_br'))
    hdus.append(fits.ImageHDU(par_err[7,:,:],name='vel_sigma_OIII_br'))
    hdus.append(fits.ImageHDU(par_err[8,:,:],name='m'))
    hdus.append(fits.ImageHDU(par_err[9,:,:],name='c'))
    hdu = fits.HDUList(hdus)

    hdu.writeto('%s/%s Extended/subcube_par_err_%s_extended_double_gauss.fits'%(destination_path_cube,obj,obj),overwrite=True)

In [16]:
def algorithm_script(obj,z,destination_path_cube = '/Volumes/Seagate',BIC_cutoff = 10,MC_loops = 2):
    (mini_data,mini_error,wave,qso_header) =  nan_free_data('%s/%s Extended/%s.extended_cube.fits'%(destination_path_cube,obj,obj))   
    (par,par_err,fitted,residuals) = FOV_modeling(obj,z,wave,mini_data,mini_error,BIC_cutoff,MC_loops)
    maps(obj,par)
    err_maps(obj,par_err)
    filecubename = '%s/%s Extended/subcube_par_%s_extended_fitted_residual_2Gauss.fits'%(destination_path_cube.obj,obj)
    store_cube(filecubename,fitted,wave,residuals,qso_header)

In [18]:
z = {"HE0040-1105":0.04196}

objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj])
 #   if __name__ == '__main__':
 #       p = Process(target=algorithm_script, args=(obj,z[obj]))
 #       p.start()
 #       p.join()



0 100
1 100
2 100


KeyboardInterrupt: 